<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/Gemma_Python_Chat_(GPC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240325%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240325T205519Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5ed0042769882a17a430166f7f69fe1b5e2cde81e1f3f7fc998cad7ea42b9fa68923dc0b79376287e7f556157282bf70c2ab6b4006ccc7d4d70915e70fbdc183f254caefc57f0cbb5206a4b7eb8120382acae444211d9886b142ee5078642532597fe79655eebb37dc61595d6e088de15b97f9c31c8cbe9f47ec92644466d569816e43a69959b73b061b64013a71d8f5396964fc25f67ada02dd0762b43ee896f63d8f7430d28d6863c8c1d1cef76a8f7136a53c62e93f5917332bbb42c8310b5546fa929faff5a5de9cb0a0c74237f37500e0257773bc15bd9bb98746e341de6f9cbfac04eff1137671e62172b9c277256a940e328649c61064b26bb13cd93f,pythonquestions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F262%2F726715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240325%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240325T205519Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D79b2fc22c81ad786ba7a9827083d58d4b1367e2ff49dcff1d31c14e0917c17d137ba7773f4975ec76f4f854d05a8ace185e1c622217db481f24e223ee7e6398aeb6c30292f5e11fe2ce739817d034e416fa56b3837a2f184a46f886bc9e144050d8585a18964c71ac0da43b7058a0a280126a6e746f0a60d8e019266d2887f1d0216f8eea0fc7a0f614abdda7764f53a8e9c2bd995f86d6aec0712d4371eaf68b376194ae2662a50eebfbfed702cfe308bbaf694857675d833d656a2fa2a29235243c1c2c05a6f68e14d6eccf8acba7c9a6880235a1ca2a3b417e2c551a0855c44bc8268ee0eadd2e6b69d57f9c0c1a4ba5742f6c68bd5d6e7fa6c3560696084,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240325%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240325T205519Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5bc4a6520d61ffa75af9027ba0aafe8144e598ed497affece772674ec842c238391d4e47090d663770f199327fda66afb2bc5d6da2e57fb3ace549ccb1916f7a147a8f4e524653e28a3fbcb70cdbdb9275602d2439caff28e73f1cb2a5760000017e744f410204c62fedf9e28902105eb8e34ef55a715b56bdbd1033e599a080aa71cfadb05eb747a4cf9e6d70d08493d396035412eceee8007f40f76d4f03ae4dea53ffff1da4db128da1ae3deb1b2e20801b9411c4c03cb1edf80a5f5eff10cd60b743d5ca73672f26d4530791234564770e5863533235e1e8b17619ebdd8ff5b3c5eb3413f31fb7563f0aaf8e858cd2b9ab9ece29d3b7b41788e2a207b88b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP 🤖 Gemma Python Chatbot

![](https://i.ibb.co/8xZNc32/Gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powereg by [Gemma 7B IT](https://blog.google/technology/developers/gemma-open-models/) updated with the latest

## Requirements

In [ ]:
#base
import os
import json
import csv
import pandas as pd
from typing import *
# model
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
# data
from datasets import load_dataset

In [ ]:
model_checkpoint = "/kaggle/input/gemma/transformers/2b/2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, torch_dtype=torch.float16).cuda()

## Data

In [ ]:
class pythonQAData:
    """
    Processes data from Questions and Answers CSV files to provide a structured Q&A format.

    Attributes:
        questions_path (str): Path to the Questions CSV file.
        answers_path (str): Path to the Answers CSV file.

    Methods:
        load_data(): Loads the CSV data into DataFrames.
        merge(): Cleans, merges, and formats the question and answer data.
        get_formatted_qa(): Returns a list of formatted question-answer strings.
    """

    def __init__(self, questions_path, answers_path):
        self.questions_path = '../input/pythonquestions/Questions.csv'
        self.answers_path = '../input/pythonquestions/Answers.csv'
        self.answers_path = '../input/pythonquestions/Tags.csv'

    def load_data(self):
        """Loads Questions and Answers data from CSV files."""
        self.df_questions = pd.read_csv(self.questions_path, encoding="ISO-8859-1", usecols=['Id', 'Score', 'Title'])
        self.df_answers = pd.read_csv(self.answers_path, encoding="ISO-8859-1", usecols=['ParentId', 'Score', 'Body'])


    def merge(self):
        """Cleans, merges, and formats the question and answer data."""
        # Filter by score
        self.df_questions = self.df_questions[self.df_questions['Score'] > 0]
        self.df_answers = self.df_answers[self.df_answers['Score'] > 0]

        # Sort and deduplicate answers
        self.df_answers = self.df_answers.sort_values('Score', ascending=False).drop_duplicates(subset=['ParentId'])

        # Merge and rename
        self.qa_data = self.df_questions.merge(self.df_answers, left_on='Id', right_on='ParentId')\
                                        .rename(columns={'Title': 'Question', 'Body': 'Answer'})[['Question', 'Answer', 'Score_x']]

    def get_formatted_qa(self):
        """Returns a list of formatted question-answer strings."""
        data = []
        for index, row in self.qa_data.iterrows():
            data.append(f"Question:\n{row['Question']}\n\nAnswer:\n{row['Answer']}")
        return data


## RAG

## Chatbot

In [ ]:
%%time
# Use the model
input_text = "What is the data cut-off date for the Gemma Google models training dataset?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=200)
print(tokenizer.decode(outputs[0]))

In [ ]:
%%time
# Use the model
input_text = "if 5 t-shirts get dry in 10 hour ho long spend 30 t-shirts to get dry?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=500)
print(tokenizer.decode(outputs[0]))

## GCP - 0.0.1

## Fine-Tunning

## GCP - 0.1.0